# Projet d'Algorithmique pour la génomique 

### Développement d'une solution de mapping de données de séquencçage à haut-débit sur un génome de référence

## I. Objectif

L'objectif de ce projet de génomique était de réussir à concevoir une stratégie de mapping de reads sur un génome donné.
Il a été nécessaire durant ce projet de prêter une attention particulière aux temps de calculs et d'alignement qui peuvent être très longs en fonction de la solution proposée. De plus, les reads peuvent comporter des erreurs de séquençages qu'il est nécessaire de prendre en considération.

### Calcul de la table des Suffixes (Difference Cover size 3)

Afin de pouvoir développer une stratégie de mapping efficace, il est nécessaire de calculer la table des suffixes de chacun des chromosomes. Cela permettra de retrouver rapidement les reads au sein des séquences. 
Il nous faut cependant utiliser une méthode particulière pour obtenir la table dans un temps acceptable. Pour cela nous utilisons, l'algorithme DC3 (DIfference Cover size 3).

In [10]:
%run DC3.py

###################################
######    DC3 Run Test    #########
###################################

Retrieving genomic data...
Genomic data retrieved !

Computing the Suffix array of the first chromosome...
Suffix array computed !

First 50 indexes of the suffix array : 
[947101, 468510, 633566, 468511, 434614, 207415, 633567, 468512, 434615, 207416, 633568, 388233, 468513, 434616, 207417, 633569, 724610, 388234, 468514, 434617, 207418, 633570, 724611, 388235, 468515, 434618, 207419, 533168, 633571, 724612, 388236, 468516, 434619, 207420, 533169, 633572, 238221, 724613, 441106, 388237, 468517, 434620, 207421, 533170, 684120, 633573, 422507, 238222, 656252, 724614]


L'algorithme de DC3 nous permet ainsi d'avoir une liste ordonnée de tous les suffixes de chaque chromosome. Afin de ne pas avoir à recalculer cette dernière à chaque nouveau mapping, nous avons décidé de stocker les données ainsi générée dans des fichiers textes. 

Grâce à cet algorithme, la complexité du calcul de la table des suffixes est linéaire dans le temps, et de cette manière nous obtenons pour chaque chromosome des résultats en moins d'une minute.

In [ ]:
import json
f = open('data.json',)
   
# returns JSON object as 
# a dictionary
data = json.load(f)

### Calcul de la BWT

En ayant ainsi obtenu les listes de suffixes par l'algorithme de DC3, nous pouvons aussi à présent calculer la Burrows Wheeler Transform (BWT) qui peut permettre de mapper les reads dans chacun des chromosomes. La BWT est simplement obtenue en cherchant le caractère précédant chacun des premiers caractères de la table des suffixes.

La BWT peut aussi être utile pour la compression de données. En effet, cette dernière à tendance à réunir les caractères similaires ensemble. Ici nous ne nous sommes pas servi de cette propriété pour compresser les données. Les résultats ont aussi été stockés dans des fichiers textes. Dans ces derniers ont aussi été stockés le rang de chaque caractère du génome. Cette information est très utile pour le mapping par la suite.

Les BWT, tables des suffixes ont été stockées dans des fichiers .json pour faciliter leur utilisation.

Mapping

Notre algorithme de alignement à partir de la BWT utilise les fonctions suivantes pour fonctionner.

In [ ]:
def rank(str,char,i,j):
    Count1=0
    k=0
    for k in range(i):
        if str[k]==char:
            Count1+=1
    Count2 = Count1
    for k in range(i,j):
        if str[k]==char:
            Count2+=1
    return Count2, Count1


La fonction rank prend en entrée un string, un caractère et 2 positions i et j et retourne le nombre d'occurences du caractère jusqu'à la position i et j.

In [ ]:
def band(str,char):
    a=np.array(sorted(str.strip()))
    return np.where(a==char)

bgA=band(S,"A")
bgT=band(S,"T")
bgG=band(S,"G")
bgC=band(S,"C")

dico={"A":bgA,"T":bgT,"G":bgG,"C":bgC}

La fonction band prend en paramètres un string et un caractère et retourne les positions dans le string trié où le caractère est présent. Pour éviter de recalculer cela à chaque appel de fonction, le résultats est stocké dans un dictionnaire.

In [ ]:
def BWAo(str, rank, read, dico): 
    index=len(read)-1 
    s = read[index] 
    
    band_start=dico[s][0][0] #donne les indices correspondant à la dernière lettre du read
    band_end=dico[s][0][-1]+1

    numDic = {"A":0, "C":1, "G":2, "T":3}
    for i in range(index-1,-1,-1): #parcours le read en partant de la fin
        if(band_start == band_end): #condition si le read n'est pas retrouvé
            pass
        s = read[i] #donne la lettre suivante à rechercher
        letterNum = numDic[s] 
        if band_start>0: #retourne les rangs de s 
            rank_top, rank_bottom = rank[band_start-1][letterNum], rank[band_end-1][letterNum] 
        else:
            rank_top, rank_bottom = rank[band_start][letterNum], rank[band_end-1][letterNum]
        a=dico[s][0][0]
        band_start=a+rank_top #nouvelle bande qui correspond au à la partie de read parcouru
        band_end=a+rank_bottom
    return band_start,band_end #retourne les positions deans la table des suffixes où le read est retrouvé


: 

In [ ]:
def kmers(str,k):
    """
    Paramètres: str:string, k:taille des kmers
    Sortie: listes des kmers dans l'ordre
    """
    l=len(str)
    b=l-l%k
    a=[]
    for i in range(0,b,k):
        a.append(str[i:i+k])
    if b!=l:
        a.append(str[b:])
    return a


La fonction kmers permet de decouper le string en kmers

En recherchant dans la table des suffixes ordonné les positions donnes par la fonction BWAo, on peut obtenir la liste des positions où le read est retrouvé. Cette liste est utilisé par la suite dans la fonction ci-dessous pour determiner la localisations des reads.

In [ ]:
def mapping(pos,S,k,kread):
    power=[]
    for j in range(len(pos)):
        err=0 #nombre d'erreurs autorisées
        #errorlist=[]
        d=0 #variable qui indique le decalage du cadre de lecture en cas d'indels
        for c in range(1,len(kread)):
            a,b=pos[j]+c*k+d,pos[j]+(c+1)*k+d #coordonnées théoriques du kmer n+1 
            if S[a:b]!=kread[c]:
                err-=1 
                if err==-1: #la boucle s'arrete si trop d'erreurs
                    break
                if c<len(kread)-1:  
                    if S[a+k+1:b+k+1]==kread[c+1]: #le kmer n+2 est retrouvé si insertion
                        d+=1
                    if S[a+k-1:b+k-1]==kread[c+1]: #le kmer n+2 est retrouvé si deletion
                        d-=1
                #errorlist.append(a)
        if err!=-1:
            power.append(pos[j])
    return power

La fonction mapping prend en paramètres une liste des positions où le premier kmer du read est retrouvé (pos), le génome (S), la taille des kmers (k) et la liste des kmers du read dans l'ordre (kread). Le principe de l'algorithme consiste à pour chaque position du premier kmer du read, de comparer la suite dans le génome au kmer du read suivant. 
La variable errorlist retourne la liste des positons des kmers qui n'ont pas été retrouvé au sein du read.
Dans le cas où une mutation/erreur de séquençage sur le premier read, la fonction devrait être relancé en partant du 2e kmers.  

La complexité temporelle de l'algorithme a été évalué avec le module cprofile, qui permet de donner les fonctions qui prennent le plus de temps. Celui indique que le mapping tourne à environ 700 itérations/secondes. La fonction mapping est celle qui occupe la majorité du temps de calcul. Cette fonction parcours la liste des positions du kmers 1 se qui est au pire de l'orde de la taille du génome, et pour chaque itérations elle parcours la liste des kmers du read en faisant des comparaisons entre string des la taille du kmers. On peut donc évaluer la complexité de la fonction mapping à environ O(n*m) où n=len(genome) et m=len(read). 

Le mapping étant une opération coûteuse en ressources, il semble intérressant d'utiliser des methodes de nettoyage des reads pour eviter d'utiliser des ressources inutilement. Par exemple, avec Trimmomatic, il est possible de retirer les séquences qui sont de faibles qualités ou encore de retirer les séquences qui correspondent à des amorces de PCR au extrêmités des reads.

### Protocole d'interpretation des résultats

Avec la listes de positions des reads sur le génome et la liste des erreurs, on peut quantifier la couverture du génome en comptant les nombres d'occurences de kmers. Cela permettrait d'identifier si une différence entre les reads et le génome est dûe à un mutation (alors le nombre de kmers différents est equivalent au nombre de kmers retrouvés). Au contraire, si un nombre faible de kmers est différent pour une position, il s'agit plus probablement d'une erreur de séquençage.